# FO-739_XML_Extraction
Code to extract JSW medial and JSW lateral with bounding box out of KOALA xml<br>


author = MV<br>
date = 2021-10-04<br>

_______________________________________

- Extract information of KOALA
- claculate classes wirh 10% JSN
- check KOALA segmentations for correctness

# Imports

In [ ]:
!nvidia-smi

In [ ]:
# this defines the GPU you are using
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
# add paths for dnn2 and labelbox-connector
import sys
sys.path.insert(1, "/srv/dnn-framework2")
sys.path.insert(1, "/srv/labelbox-connector")

In [ ]:
#general
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import logging
import cv2
import copy

#tensorflow
import tensorflow as tf


import shutil
import os

import pydicom

from lxml import etree
import xml.etree.ElementTree as ET
from framework.data_objects import BoundingBox, PointList2D, DicomImage
import ast
from framework.inferences import Inference

In [ ]:
#configs
%matplotlib inline
logging.basicConfig(format='%(asc' 'time)s %(name)-25s %(level' 'name)-8s %(message)s')
logging.getLogger().setLevel(logging.INFO) # you change this to logging.DEBUG to get more logging information

# XML Extraction and Data Preparation 

## Extraction min. JSW and BBox coords

In [ ]:
# create dataframe with minimal JSW of left and right knee on medial and lateral side

df = pd.DataFrame(columns = {'ID','Laterality','JSW_MIN_MED','JSW_MIN_LAT','coords_bbox','rotation'})
path = "/mnt/temporaer/MV/KOALA_output/xml/"

for i, f in enumerate(os.listdir(path)):
    print(i)
    try:
        xm = etree.parse(os.path.join(path,f))
        for idx in [0,1]:
            # extraction Laterality
            if idx == 0:
                s = 'R'
            else:
                s = 'L'
            # extraction ID
            ID = xm.xpath('/Analysis/DICOM/DCM0008/SOPInstanceUID')[0].text
            # extraction medial and lateral JSW_MIN
            x_med = xm.xpath(f"/Analysis/Region[id='{idx}']/MeasurementBoard[id='JSx']/JSx[location='MED']/JSW_MIN/valueCalc")[0].text
            x_lat = xm.xpath(f"/Analysis/Region[id='{idx}']/MeasurementBoard[id='JSx']/JSx[location='LAT']/JSW_MIN/valueCalc")[0].text
            
            # extraction boundingbox
            a = xm.xpath(f"/Analysis/Region[id='{idx}']/BoundingBox/coordinates")[0].text
            coords = [(a.split(' ')[0],a.split(' ')[1]),(a.split(' ')[2],a.split(' ')[3]),(a.split(' ')[4],a.split(' ')[5]),(a.split(' ')[6],a.split(' ')[7])]
            # check if bounding box is rotated 
            if coords[0][1]==coords[1][1]:
                rot = 0
            else:
                rot = 1
            
            row = {
                'JSW_MIN_MED' :  x_med,
                'JSW_MIN_LAT' : x_lat,
                'ID': ID,
                'Laterality': s,
                'coords_bbox': coords,
                'rotation': rot
            }
            df = df.append(row, ignore_index = True)
    except: 
        print('error at',i)
        continue
        
df[['ID','Laterality','JSW_MIN_MED','JSW_MIN_LAT','coords_bbox','rotation']]    

## Extraction Osteophytes, scerosis and KL

In [ ]:
df = pd.read_csv('/srv/temp(1).csv')

In [ ]:
df_new = pd.DataFrame(columns={'ID','Laterality','osteophytes','sclerosis','KL-grade'})
path = "/mnt/temporaer/MV/KOALA_output/xml/"
for i, f in enumerate(os.listdir(path)):
    print(i)
    try:
        xm = etree.parse(os.path.join(path,f))
        for idx in [0,1]:
            # extraction Laterality
            if idx == 0:
                l = 'R'
            else:
                l = 'L'
                
            # extraction ID
            ID = xm.xpath('/Analysis/DICOM/DCM0008/SOPInstanceUID')[0].text
            
            # extraction osteophyte score
            o = xm.xpath(f"/Analysis/Region[id='{idx}']/ScoreBoard[id='OARSI']/score[id='osteophytes']/active/value")[0].text
            s = xm.xpath(f"/Analysis/Region[id='{idx}']/ScoreBoard[id='OARSI']/score[id='sclerosis']/active/value")[0].text
            kl = xm.xpath(f"/Analysis/Region[id='{idx}']/ScoreBoard[id='KL']/score[id='grading']/active/value")[0].text
            
            row={
                'ID': ID,
                'Laterality': l,
                'osteophytes': o,
                'sclerosis': s,
                'KL-grade': kl
            }
            
            df_new = df_new.append(row, ignore_index = True)
    except:
        print('error at', i)
    

In [ ]:
df_new.to_csv('/srv/XMLExtrKLOsteoScleroAll.csv')

In [ ]:
df_orig = pd.read_csv('/srv/JSW_all_2_caroline.csv')

df0 = df_new.rename(columns={'ID':'ID_a','osteophytes':'osteophytes_a', 'sclerosis':'sclerosis_a', 'KL-grade':'KL-grade_a'})
df1 = df_orig.merge(df0, on = ['ID_a','Laterality'], how = 'left')

df2 = df_new.rename(columns={'ID':'ID_b','osteophytes':'osteophytes_b', 'sclerosis':'sclerosis_b', 'KL-grade':'KL-grade_b'})
df3 = df1.merge(df2, on = ['ID_b','Laterality'], how = 'left')

In [ ]:
df3.to_csv('/srv/temp.csv')

In [ ]:
# sclerosis, oseophytosis and KL for a) knee at year 0 and b) for image at year 1 or year 2

path = "/mnt/temporaer/MV/KOALA_output/xml/"

df_orig = pd.read_csv('/srv/JSW_all_2_caroline.csv')
df_new = df_orig.copy()
df_new['osteophytes_a']=''
df_new['sclerosis_a']=''
df_new['KL-grade_a']=''

df_new['osteophytes_b']=''
df_new['sclerosis_b']=''
df_new['KL-grade_b']=''

for i, row in df_orig.iterrows():
    print(i, end='\r')
    i = i+10
    try:
        xm_a = etree.parse(os.path.join(path, df_orig['ID_a'][i],'.xml'))
        xm_b = etree.parse(os.path.join(path, df_orig['ID_b'][i],'.xml'))

        # extraction Laterality
        if df_orig['Laterality'][i] == 'R':
            l = 0
        else:
            l = 1

        # extraction osteophyte score
        oa = xm_a.xpath(f"/Analysis/Region[id='{l}']/ScoreBoard[id='OARSI']/score[id='osteophytes']/active/value")[0].text
        sa = xm_a.xpath(f"/Analysis/Region[id='{l}']/ScoreBoard[id='OARSI']/score[id='sclerosis']/active/value")[0].text
        kla = xm_a.xpath(f"/Analysis/Region[id='{l}']/ScoreBoard[id='KL']/score[id='grading']/active/value")[0].text
        
        df_new['osteophytes_a'][i]=oa
        df_new['sclerosis_a'][i]=sa
        df_new['KL-grade_a'][i]=kla
        
        ob = xm_b.xpath(f"/Analysis/Region[id='{l}']/ScoreBoard[id='OARSI']/score[id='osteophytes']/active/value")[0].text
        sb = xm_b.xpath(f"/Analysis/Region[id='{l}']/ScoreBoard[id='OARSI']/score[id='sclerosis']/active/value")[0].text
        klb = xm_b.xpath(f"/Analysis/Region[id='{l}']/ScoreBoard[id='KL']/score[id='grading']/active/value")[0].text
        
        df_new['osteophytes_b'][i]= ob
        df_new['sclerosis_b'][i]= sb
        df_new['KL-grade_b'][i]= klb

    except:
        print('error at', i)
    

# Filter images with at least two year interval out of Masterfile

In [ ]:
# load master file
df_master = pd.read_csv('/srv/Master_dataset_JSN.csv')
df_tmp = df_master.copy()
df_tmp['tmp'] =''

# Change visit from V00 to 0 (number to calculate)
df_tmp[['VISIT']] = df_tmp[['VISIT']].replace(
    ['V00','V01','V02','V03','V04','V05','V06','V07','V08','V09','V10'],
    [0,1,2,3,4,5,6,7,8,9,10])
df_tmp = df_tmp.astype({'Patient ID': 'str'})

# split into left and right dataframe dfL & dfR
dfL = df_tmp.loc[df_tmp['Laterality']=='L'].reset_index()
dfR = df_tmp.loc[df_tmp['Laterality']=='R'].reset_index()


# Dataframe with 'a' as year 0 and 'b' as year 1 or 2
df_img_L = pd.DataFrame(columns={'Study','Patient ID','Image_a','Image_b','ID_a','ID_b','VISIT_a','VISIT_b', 'KL_a','KL_b'})

# left knees
for idx, row in dfL.iterrows():
    print(idx)
    if idx < 40027:
        i = idx+1 # to have to following rows i and idx 
        if dfL['Patient ID'][i] == dfL['Patient ID'][idx]: # make sure its same Patient 
            if 1<= (dfL['VISIT'][i] - dfL['VISIT'][idx]) <= 2: # Visit with 1 or 2 year interval 
                
                row = {
                    'ID_a': dfL['ID'][idx],
                    'ID_b': dfL['ID'][i],
                    'Image_a': dfL['img_path'][idx],
                    'Image_b': dfL['img_path'][i],
                    'VISIT_a': dfL['VISIT'][idx],
                    'VISIT_b': dfL['VISIT'][i],
                    'KL_a' : dfL['KL'][idx],
                    'KL_b' : dfL['KL'][i],
                    'Patient ID': dfL['Patient ID'][idx],
                    'Study' : dfL['Study'][idx]
                }
                df_img_L = df_img_L.append(row, ignore_index = True)

            else:
                continue
        else: 
            continue
    else:
        break
        
        
df_img_R = pd.DataFrame(columns={'Study','Patient ID','Image_a','Image_b','ID_a','ID_b','VISIT_a','VISIT_b', 'KL_a','KL_b'})

# right side 
for idx, row in dfR.iterrows():
    print(idx)
    if idx < 40021:
        i = idx+1
        if dfR['Patient ID'][i] == dfR['Patient ID'][idx]:
            if 1<= (dfR['VISIT'][i] - dfR['VISIT'][idx]) <= 2:
                
                row = {
                    'ID_a': dfR['ID'][idx],
                    'ID_b': dfR['ID'][i],
                    'Image_a': dfR['img_path'][idx],
                    'Image_b': dfR['img_path'][i],
                    'VISIT_a': dfR['VISIT'][idx],
                    'VISIT_b': dfR['VISIT'][i],
                    'KL_a' : dfR['KL'][idx],
                    'KL_b' : dfR['KL'][i],
                    'Patient ID': dfR['Patient ID'][idx],
                    'Study' : dfR['Study'][idx]
                    
                }
                df_img_R = df_img_R.append(row, ignore_index = True)

            else:
                continue
        else: 
            continue
    else:
        break
        
print(len(df_img_R))
print(len(df_img_L))

In [ ]:
    ## LEFT knees ## 

df_img_L['Laterality']='L'

# Add JSW_MIN and JSW reduction of image_a (year 0 )
df_t = df.rename(columns = {'ID':'ID_a'}) # change column name to merge master csv with JSW MIN on ID

df2 = df_img_L.merge(df_t, on = ['ID_a','Laterality'], how = 'inner')
df2 = df2.rename(columns = {'JSW_MIN_MED':'JSW_MIN_MED_a',
                            'JSW_MIN_LAT': 'JSW_MIN_LAT_a',
                            'coords_bbox':'coords_bbox_a',
                            'rotation':'rotation_a'})

# Add JSW_MIN and JSW reduction of image_b (year 1 or 2)
df_t = df.rename(columns = {'ID':'ID_b'})

df3 = df2.merge(df_t, on = ['ID_b','Laterality'], how = 'inner')
df3 = df3.rename(columns = {'JSW_MIN_MED':'JSW_MIN_MED_b', 
                            'JSW_MIN_LAT': 'JSW_MIN_LAT_b',
                            'coords_bbox':'coords_bbox_b',
                            'rotation':'rotation_b'})

    ## RIGHT knees ##

df_img_R['Laterality']='R'
# Add JSW_MIN and JSW reduction of image_a (year 0 )
df_t = df.rename(columns = {'ID':'ID_a'})

df4 = df_img_R.merge(df_t, on = ['ID_a','Laterality'], how = 'inner')
df4 = df4.rename(columns = {'JSW_MIN_MED':'JSW_MIN_MED_a', 
                            'JSW_MIN_LAT': 'JSW_MIN_LAT_a',
                            'coords_bbox':'coords_bbox_a',
                            'rotation':'rotation_a'})

# Add JSW_MIN and JSW reduction of image_b (year 1 or 2)
df_t = df.rename(columns = {'ID':'ID_b'})

df5 = df4.merge(df_t, on = ['ID_b','Laterality'], how = 'inner')
df5 = df5.rename(columns = {'JSW_MIN_MED':'JSW_MIN_MED_b', 
                            'JSW_MIN_LAT': 'JSW_MIN_LAT_b',
                            'coords_bbox':'coords_bbox_b',
                            'rotation':'rotation_b'})


    ## Append R and L ##
    
df6 = df3.append(df5, ignore_index = True)
df6 = df6[['Patient ID','Laterality','ID_a', 'VISIT_a','Image_a',
           'coords_bbox_a','rotation_a','ID_b','VISIT_b','Image_b',
           'coords_bbox_b','rotation_b','JSW_MIN_MED_a','JSW_MIN_MED_b',
           'JSW_MIN_LAT_a','JSW_MIN_LAT_b','KL_a','KL_b'
          ]]
df6.sort_values(by = ['Patient ID','Laterality'], ignore_index = True)

    ## Calculate 10% JSR/year
    
df6['class_MED']=''
df6['class_LAT']=''

In [ ]:
df6.to_csv('/srv/JSW_all.csv')

# Calculate JSN/year and Classify into slow and fast progressors (10% JSN)
 Exclusion criteria KL0-0, KL1-1 and KL4 at basleine

In [ ]:
df6 = pd.read_csv('/srv/JSW_all.csv')

for i, row in df6.iterrows(): 
    print(i)
    
    # calculate difference btw. JSW of image a and image b 
    diff_MED = float(df6.JSW_MIN_MED_a[i]) - float(df6.JSW_MIN_MED_b[i]) 
    diff_LAT = float(df6.JSW_MIN_LAT_a[i]) - float(df6.JSW_MIN_LAT_b[i])
    
    ## MED
    if 0 > diff_MED > -0.4: # tolerance of -0.4mm == difference is 0 
        diff_MED = 0 
    else:
        s=1
    
    if (diff_MED >= 0) & (float(df6.JSW_MIN_MED_a[i]) > 0): # diff must be + and baseline JSW must be > 0 (0 anyway KL4 --> exclusion)
        reduction_MED = diff_MED/float(df6.JSW_MIN_MED_a[i]) # calculate percentage of baseline 

        if (reduction_MED < 0.1)| (diff_MED < 0.4) :
            df6.class_MED[i] = 0 # less than 10% --> slow progr. (class 0)
        else:
            df6.class_MED[i] = 1 # else --> fast progr. (class 1)
    else:
        df6.class_MED[i] = np.nan
        
    ## LAT
    if 0 > diff_LAT > -0.4:
        diff_LAT = 0 
    else:
        s=1
        
    if (diff_LAT >= 0) & (float(df6.JSW_MIN_LAT_a[i]) > 0):
        reduction_LAT = diff_LAT/float(df6.JSW_MIN_LAT_a[i])

        if (reduction_LAT < 0.1)| (diff_LAT < 0.4):
            df6.class_LAT[i] = 0
        else:
            df6.class_LAT[i] = 1
    else:
        df6.class_LAT[i] = np.nan

In [ ]:
df7 = df6.copy()
df7.to_csv('/srv/JSW_all_2.csv')


## Extract images of exclusion criteria

In [ ]:
df7 = pd.read_csv('/srv/JSW_all_2.csv')
df_x = df7.dropna(subset=['class_MED','class_LAT'], how = 'all')
print(len(df_x))

# if KL_b is 0 
df_y = df_x[df_x.KL_b != 0]
print(len(df_y))

# if KL stays 1
df_z = df_y.drop(df_y[(df_y['KL_a'] ==1) & (df_y['KL_b'] ==1)].index)
print(len(df_z))

# id KL_a is 4
df8 = df_z[df_z.KL_a != 4]

# add one general class combining class_MED and class_LAT
df8['class']=''
for i, row in df8.iterrows():
    if (df8['class_MED'][i]==1) | (df['class_LAT'][i]==1):
        df8['class'][i]=1
    else:
        df8['class'][i]=0
        
print('Length of Dataset with applied exclusion criteria:',len(df8))
print('Number of right knees:', len(df8[df8.Laterality == 'R']))
print('Number of left knees:', len(df8[df8.Laterality == 'L']))

print("Number of medial slow progressors:",len(df8.loc[df8.class_MED == 0]))
print("Number of medial fast progressors:",len(df8.loc[df8.class_MED == 1]))

print("Number of lateral slow progressors:",len(df8.loc[df8.class_LAT == 0]))
print("Number of lateral fast progressors:",len(df8.loc[df8.class_LAT == 1]))

print('Class 0:',len(df8.loc[df8['class'] == 0]))
print('Class 1:',len(df8.loc[df8['class'] == 1]))



In [ ]:
# contains Image a and Image b with class_MED and class_LAT and general class 
# Exclusion criteria are applied
df8.to_csv('/srv/JSW_excluded.csv')

In [ ]:
# Add class to master csv
df_master = pd.read_csv('/srv/Master_dataset_JSN.csv')

df7_tmp = df8.rename(columns = {'ID_a':'ID'})
df_c = df_master.merge(df7_tmp[['ID','Laterality','coords_bbox_a','class_MED','class_LAT','class']], on = ['ID','Laterality'], how = 'right')


In [ ]:
df_c.to_csv('/srv/JSN_pred_clf.csv')

## Eliminate bad segmented images 

FailedKOALAxml.csv: contains images which were bad segemented by KOALA

In [ ]:
c = pd.read_csv('/srv/JSN_pred_clf.csv')
a = pd.read_csv('/srv/FailedKOALAxml.csv')

a = a.replace(['l','r'], ['L','R']).rename(columns={'Side': 'Laterality'})
m = a.merge(c, on = ['Laterality', 'ID'], how = 'inner')

indices = []
for i, row in m.iterrows():
    index = m['Unnamed: 0'][i]
    indices.append(index)
    

d = c.drop(indices, axis = 0).reset_index()
e = d.drop(columns=['index','Unnamed: 0', 'Unnamed: 0.1'])

e.to_csv('/srv/JSN_pred_clf.csv')

## Add column of contralateral knee OA

In [ ]:
df = pd.read_csv('/srv/JSN_pred_clf.csv')
df['other_knee_AKOA']=''
print(len(df))
df.sort_values(by = 'ID')
mask = df.duplicated(subset=['ID'], keep = False)

# keep only bilateral images 
df_d = df[mask].reset_index()
df_d
print(len(df_d))

# add other knee fast progressor column
for idx, row in df_d.iterrows():

    print(idx, end = '\r')
    i = idx+1 # idx = first row, i = following row 

    if df_d['ID'][i] == df_d['ID'][idx]:
        if df_d['class'][idx]==1:
            df_d['other_knee_KOA'][i]= 1
        else: 
            df_d['other_knee_KOA'][i]= 0
            
        if df_d['class'][i]==1:
            df_d['other_knee_KOA'][idx]= 1
        else: 
            df_d['other_knee_KOA'][idx]= 0
    else:
        continue
        
df_d 

In [ ]:
df_d[:20]

In [ ]:
df = pd.read_csv('/srv/JSN_pred_clf.csv')
df = df.rename(columns = {'img_path':'dicom_img_path'})


In [ ]:
#df.to_csv('/srv/JSN_pred_clf_new.csv')

# Draw JSW Boxes to check Segmentations

In [ ]:
def fake_jsw_box(xml_path, loc="MED", side="R"):
    xm = etree.parse(xml_path)
    region_id = 0 if side=="R" else 1
    ptlist = []
    for i in range(4):
        xpt = f"/Analysis/Region[id='{region_id}']/MeasurementBoard[id='JSx']/JSx[location='{loc}']/JSW{i}/coordinates"
        xy = ast.literal_eval(xm.xpath(xpt)[0].text.replace(" ",","))
        coords = [ xy[i:i + 2] for i in range(0,len(xy),2)]
        ptlist.extend(coords)

    ptlist = PointList2D(ptlist)

    c = ptlist.center.data
    dx = ptlist.dx
    dy = int(1.5 * dx)
    #print(c, dx, dy)
    return BoundingBox.from_center_height_width(c, dy, dx)


def get_jsw_segmentation(xml_path, dicom_path, model, loc="MED", side="R"):
    bbox = fake_jsw_box(xml_path, loc, side)
    dicom_image = DicomImage(dicom_path).load()
    cut_image = bbox.cut(dicom_image.array)

    if side == "R" and loc == "LAT" or side == "L" and loc == "MED":
        cut_image = np.fliplr(cut_image)

    return cut_image, model.infer(cut_image)[0][0]    


def plot_jsw(img, segm):
    f, ax = plt.subplots(1,1)
    ax.imshow(img,cmap="gray")
    segm.draw_on(ax)
    plt.show()

    
def plot_all_four(xml_path, dicom_path, model):
    f, ax = plt.subplots(1, 4)
    ctr = 0
    xm = etree.parse(xml_path)
    for side in {"R", "L"}:
        region_id = 0 if side=="R" else 1
        for loc in {"LAT", "MED"}:
            i, s = get_jsw_segmentation(xml_path, dicom_path, model, loc, side)    
            fig = ax[ctr].imshow(i, cmap="gray")
            #print(region_id, loc,xm.xpath(f"/Analysis/Region[id='{region_id}']/MeasurementBoard[id='JSx']/JSx[location='{loc}']/JSW_MIN/valueCalc")[0].text)
            #print(region_id, loc,xm.xpath(f"/Analysis/Region[id='{region_id}']/MeasurementBoard[id='JSx']/JSx[location='{loc}']/JSW_MIN/valueCalc")[0].text)
            fig.axes.get_xaxis().set_visible(False)
            fig.axes.get_yaxis().set_visible(False)
            s.draw_on(ax[ctr])
            ctr += 1       
    plt.show()

In [ ]:
jsw_model = Inference(
    "/mnt/swe/VArchiv/IAS/resources/KOALA/JNET/V2.11/protobufs/KneeJsw.xml",
    "/mnt/swe/VArchiv/IAS/resources/KOALA/JNET/V2.11/protobufs/KneeJSW.pb")

In [ ]:
# plot all images (duplicates were dropped)
df8 = pd.read_csv('/srv/JSW_excluded.csv')
tmpR = df8.drop_duplicates(subset='ID_a')
#tmpR = df7.loc[((df7['class_MED'] == 1) | (df7['class_LAT'] == 1)) & (df7['Laterality'] == 'R')]

for i, row in tmpR.iterrows():  
    print('Image_a',tmpR['ID_a'][i],tmpR['Laterality'][i],'class:',tmpR['class'][i])
    plot_all_four(
        os.path.join("/mnt/temporaer/MV/KOALA_output/xml",tmpR.loc[i]["ID_a"]+".xml"),
        tmpR.loc[i]["Image_a"],jsw_model)
    print('Image_b',tmpR['ID_b'][i],tmpR['Laterality'][i],'class:',tmpR['class'][i])
    plot_all_four(
        os.path.join("/mnt/temporaer/MV/KOALA_output/xml",tmpR.loc[i]["ID_b"]+".xml"),
        tmpR.loc[i]["Image_b"],jsw_model)